# Laboratorio: Agente de Pastelería con LlamaIndex y Qwen2.5 (HuggingFace)

**Objetivo:** Crear un agente conversacional que atiende pedidos de una pastelería
usando un modelo local (Qwen2.5-7B-Instruct) cuantizado con HuggingFace Transformers.

**Características:**
- Modelo cuantizado 4-bit (corre en Colab con GPU T4)
- HuggingFace Transformers + bitsandbytes
- Vector store para el catálogo de productos
- Tools personalizadas (cálculo de precio, registro de pedidos, notificación Telegram)
- Ciclo ReAct completo (Thought → Action → Observation)

## 1. Setup e Instalación de Dependencias

In [1]:
# Instalar dependencias
print("📦 Instalando dependencias...")

# Transformers y cuantización
!pip install -q transformers accelerate bitsandbytes

# LlamaIndex core y componentes
!pip install -q llama-index-core llama-index-llms-huggingface llama-index-embeddings-huggingface

# Utilidades
!pip install -q sentence-transformers requests

print("✓ Dependencias instaladas")

📦 Instalando dependencias...
✓ Dependencias instaladas


## 2. Verificar GPU disponible

In [2]:
import torch

if torch.cuda.is_available():
    print(f"✓ GPU detectada: {torch.cuda.get_device_name(0)}")
    print(f"  VRAM disponible: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    USE_GPU = True
else:
    print("⚠ No se detectó GPU. El modelo se ejecutará en CPU (será muy lento)")
    print("  En Google Colab: Runtime > Change runtime type > GPU")
    USE_GPU = False

✓ GPU detectada: Tesla T4
  VRAM disponible: 14.7 GB


## 3. Configurar Telegram Bot (Opcional)

**Instrucciones:**
1. Abre Telegram y busca @BotFather
2. Envía `/newbot` y sigue las instrucciones
3. Copia el **token** que te da
4. Envía un mensaje a tu bot
5. Ve a `https://api.telegram.org/bot<TU_TOKEN>/getUpdates` para obtener tu **chat_id**

Si no quieres usar Telegram, deja las variables vacías.

In [3]:
# Configuración de Telegram (opcional)
TELEGRAM_BOT_TOKEN = ""  # Ej: "123456789:ABCdefGHIjklMNOpqrsTUVwxyz"
TELEGRAM_CHAT_ID = ""     # Ej: "123456789"

# Validar configuración
USE_TELEGRAM = bool(TELEGRAM_BOT_TOKEN and TELEGRAM_CHAT_ID)

if USE_TELEGRAM:
    print("✓ Notificaciones de Telegram activadas")
else:
    print("⚠ Notificaciones de Telegram desactivadas")
    print("  Configura TELEGRAM_BOT_TOKEN y TELEGRAM_CHAT_ID para activar")

✓ Notificaciones de Telegram activadas


## 4. Cargar Modelo LLM con HuggingFace Transformers

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings
import torch

print("🚀 Cargando modelo Qwen2.5-7B-Instruct...")

# Nombre del modelo en HuggingFace
model_id = "Qwen/Qwen2.5-7B-Instruct"

# Configuración de cuantización 4-bit
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,                      # Cargar en 4-bit
    bnb_4bit_compute_dtype=torch.float16,   # Tipo de datos para cómputo
    bnb_4bit_quant_type="nf4",              # Tipo de cuantización (NormalFloat4)
    bnb_4bit_use_double_quant=True,         # Doble cuantización para ahorrar más memoria
)

# Cargar tokenizer
print("  Cargando tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Cargar modelo cuantizado
print("  Cargando modelo cuantizado (esto puede tomar 1-2 minutos)...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config if USE_GPU else None,
    device_map="auto" if USE_GPU else None,
    trust_remote_code=True,
    torch_dtype=torch.float16 if USE_GPU else torch.float32,
)

print(f"✓ Modelo cargado correctamente")
print(f"  Dispositivo: {model.device}")
print(f"  Memoria GPU usada: {torch.cuda.memory_allocated() / 1024**3:.2f} GB" if USE_GPU else "")

# Configurar LLM para LlamaIndex
llm = HuggingFaceLLM(
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    generate_kwargs={
        "temperature": 0.1,
        "top_p": 0.9,
        "do_sample": True,
    },
    device_map="auto" if USE_GPU else None,
)

# Configurar como LLM global
Settings.llm = llm

print("✓ Modelo configurado en LlamaIndex")

🚀 Cargando modelo Qwen2.5-7B-Instruct...
  Cargando tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


  Cargando modelo cuantizado (esto puede tomar 1-2 minutos)...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✓ Modelo cargado correctamente
  Dispositivo: cuda:0
  Memoria GPU usada: 5.18 GB
✓ Modelo configurado en LlamaIndex


## 5. Configurar Embeddings Locales para el Catálogo

In [5]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

print("📚 Cargando modelo de embeddings...")

# Embeddings multilingües (soporta español)
embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    device="cuda" if USE_GPU else "cpu"
)

Settings.embed_model = embed_model

print("✓ Modelo de embeddings cargado")

📚 Cargando modelo de embeddings...
✓ Modelo de embeddings cargado


## 6. Crear Catálogo de Productos (Vector Store)

In [6]:
from llama_index.core import VectorStoreIndex, Document

print("🍰 Creando catálogo de productos...")

# Catálogo de productos de la pastelería
productos = [
    Document(
        text="Torta de chocolate. Tamaños disponibles: 20cm para 8-10 personas a $25000, "
             "25cm para 12-15 personas a $35000, 30cm para 18-20 personas a $45000. "
             "Decoraciones: básica incluida, decoración premium cuesta $8000 adicionales."
    ),
    Document(
        text="Torta red velvet. Tamaños: 20cm para 8-10 personas a $28000, "
             "25cm para 12-15 personas a $38000. Relleno de queso crema incluido. "
             "Opción de relleno con frutas por $5000 adicionales."
    ),
    Document(
        text="Torta tres leches. Solo disponible en tamaño 25cm para 12-15 personas a $32000. "
             "Decoración con frutas frescas incluida."
    ),
    Document(
        text="Cupcakes. Docena normal (decoración simple) a $12000. "
             "Docena premium con decoración especial a $18000. "
             "Sabores disponibles: vainilla, chocolate, red velvet."
    ),
    Document(
        text="Cheesecake. Tamaño único 23cm para 10-12 personas a $30000. "
             "Sabores: natural, frutos rojos, maracuyá. "
             "Cada sabor adicional (capa doble) cuesta $6000 más."
    ),
    Document(
        text="Brownies. Caja de 6 unidades a $8000, caja de 12 unidades a $15000. "
             "Variedades: chocolate puro, con nueces, con chips de chocolate blanco."
    ),
    Document(
        text="Alfajores artesanales. Docena a $10000. "
             "Rellenos: dulce de leche tradicional, dulce de leche con coco, chocolate."
    ),
    Document(
        text="Pie de limón. Tamaño 23cm para 8-10 personas a $22000. "
             "Base de galleta, relleno de limón natural, merengue italiano tostado."
    ),
    Document(
        text="Extras y accesorios: Velas numéricas $2000 por unidad. "
             "Topper personalizado $5000. Mensaje escrito en chocolate $3000. "
             "Caja de regalo premium $4000."
    ),
]

# Crear índice vectorial
print("  Indexando productos...")
index = VectorStoreIndex.from_documents(productos)

# Query engine para consultar catálogo
query_engine = index.as_query_engine(similarity_top_k=3)

print("✓ Catálogo de productos indexado")

🍰 Creando catálogo de productos...
  Indexando productos...
✓ Catálogo de productos indexado


## 7. Definir Tools del Agente

In [13]:
from llama_index.core.tools import FunctionTool
import json
from datetime import datetime
import requests


# Tool 1: Consultar catálogo de productos
def consultar_catalogo(consulta: str) -> str:
    """
    Busca información sobre productos en el catálogo de la pastelería.

    Args:
        consulta (str): Pregunta sobre productos, precios, tamaños, sabores, etc.
                       Ejemplos: "¿Qué tortas tienen?", "precio del cheesecake",
                                "cupcakes disponibles"

    Returns:
        str: Información relevante del catálogo
    """
    try:
        response = query_engine.query(consulta)
        return str(response)
    except Exception as e:
        return f"Error al consultar el catálogo: {str(e)}"


# Tool 2: Calcular precio total del pedido
def calcular_precio(items: str) -> str:
    """
    Calcula el precio total de un pedido.

    Args:
        items (str): Lista de items separados por coma.
                     Formato: "torta chocolate 25cm, decoracion premium, velas"
                     Ejemplos válidos:
                     - "torta chocolate 20cm, velas"
                     - "cheesecake, topper personalizado"
                     - "cupcakes docena premium, caja regalo"

    Returns:
        str: Desglose de precios y total en formato legible
    """
    # Mapeo de productos a precios (en pesos chilenos)
    precios = {
        # Tortas de chocolate
        "torta chocolate 20cm": 25000,
        "torta chocolate 25cm": 35000,
        "torta chocolate 30cm": 45000,
        # Tortas red velvet
        "torta red velvet 20cm": 28000,
        "torta red velvet 25cm": 38000,
        # Otras tortas
        "torta tres leches": 32000,
        "torta tres leches 25cm": 32000,
        "pie de limon": 22000,
        "pie limon": 22000,
        # Cheesecake
        "cheesecake": 30000,
        "cheesecake 23cm": 30000,
        "cheesecake sabor extra": 6000,
        # Cupcakes
        "cupcakes docena normal": 12000,
        "cupcakes docena": 12000,
        "cupcakes normal": 12000,
        "cupcakes docena premium": 18000,
        "cupcakes premium": 18000,
        # Brownies
        "brownies 6": 8000,
        "brownies caja 6": 8000,
        "brownies 12": 15000,
        "brownies caja 12": 15000,
        "brownies docena": 15000,
        # Alfajores
        "alfajores": 10000,
        "alfajores docena": 10000,
        # Decoraciones y extras
        "decoracion premium": 8000,
        "decoración premium": 8000,
        "relleno frutas": 5000,
        "velas": 2000,
        "velas numericas": 2000,
        "vela": 2000,
        "topper": 5000,
        "topper personalizado": 5000,
        "mensaje chocolate": 3000,
        "caja regalo": 4000,
        "caja regalo premium": 4000,
    }

    total = 0
    items_list = [item.strip().lower() for item in items.split(",")]
    desglose = []

    for item in items_list:
        encontrado = False
        for producto, precio in precios.items():
            if producto in item:
                total += precio
                desglose.append(f"  • {producto.title()}: ${precio:,} CLP")
                encontrado = True
                break

        if not encontrado:
            desglose.append(f"  • {item} (producto no encontrado en catálogo)")

    resultado = "Desglose del pedido:\n"
    resultado += "\n".join(desglose)
    resultado += f"\n\n{'='*40}\n"
    resultado += f"TOTAL: ${total:,} CLP\n"
    resultado += f"{'='*40}"

    return resultado


# Tool 3: Registrar pedido
def registrar_pedido(nombre: str, telefono: str, direccion: str, items: str, total: str) -> str:
    """
    Registra un nuevo pedido en el sistema y opcionalmente envía notificación a Telegram.

    Args:
        nombre (str): Nombre completo del cliente (ej: "María González")
        telefono (str): Teléfono de contacto, preferiblemente formato chileno
                       (ej: "+56912345678" o "912345678")
        direccion (str): Dirección de entrega completa con comuna
                        (ej: "Av. Providencia 1234, Providencia")
        items (str): Items del pedido separados por coma
                    (ej: "torta chocolate 25cm, decoracion premium")
        total (str): Precio total del pedido
                    (ej: "$45000" o "45000")

    Returns:
        str: Confirmación con número de pedido y estado de notificación
    """
    # Generar ID de pedido
    pedido_id = datetime.now().strftime("%Y%m%d%H%M%S")

    pedido = {
        "id": pedido_id,
        "timestamp": datetime.now().isoformat(),
        "cliente": {
            "nombre": nombre,
            "telefono": telefono,
            "direccion": direccion
        },
        "items": items,
        "total": total,
        "estado": "pendiente_pago"
    }

    # Guardar en JSON local
    try:
        with open("pedidos.json", "r", encoding="utf-8") as f:
            pedidos = json.load(f)
    except FileNotFoundError:
        pedidos = []

    pedidos.append(pedido)

    with open("pedidos.json", "w", encoding="utf-8") as f:
        json.dump(pedidos, f, indent=2, ensure_ascii=False)

    # Enviar notificación a Telegram (si está configurado)
    if USE_TELEGRAM:
        mensaje_telegram = f"""
🍰 *NUEVO PEDIDO - Pastelería Dulce Tentación*

📋 *Pedido:* #{pedido_id}
👤 *Cliente:* {nombre}
📞 *Teléfono:* {telefono}
📍 *Dirección:* {direccion}

🛒 *Items:*
{items}

💰 *Total:* {total}

⏰ *Fecha:* {datetime.now().strftime("%d/%m/%Y %H:%M")}
📊 *Estado:* Pendiente de pago
        """.strip()

        try:
            url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
            payload = {
                "chat_id": TELEGRAM_CHAT_ID,
                "text": mensaje_telegram,
                "parse_mode": "Markdown"
            }
            response = requests.post(url, json=payload, timeout=5)

            if response.status_code == 200:
                telegram_status = "✓ Notificación enviada a Telegram"
            else:
                telegram_status = f"⚠ Error al enviar a Telegram: {response.status_code}"
        except Exception as e:
            telegram_status = f"⚠ Error al enviar a Telegram: {str(e)}"
    else:
        telegram_status = "(Telegram no configurado)"

    confirmacion = f"""
✓ Pedido registrado exitosamente

Número de pedido: {pedido_id}
Cliente: {nombre}
Total: {total}

{telegram_status}

El pedido ha sido guardado en el sistema.
    """.strip()

    return confirmacion


# Tool 4: Obtener datos para transferencia bancaria
def obtener_datos_transferencia() -> str:
    """
    Retorna la información bancaria de la pastelería para realizar transferencias.

    Returns:
        str: Datos bancarios completos incluyendo banco, tipo de cuenta, número,
             RUT, y instrucciones de pago
    """
    return """
╔════════════════════════════════════════════╗
║   DATOS PARA TRANSFERENCIA BANCARIA       ║
╚════════════════════════════════════════════╝

🏦 Banco: Banco Estado
📊 Tipo: Cuenta Corriente
🔢 Número: 12345678-9
🆔 RUT: 12.345.678-9
👤 Nombre: Pastelería Dulce Tentación SpA

📧 Email: pagos@dulcetentacion.cl
💬 WhatsApp: +56 9 1234 5678

⚠️  IMPORTANTE:
   • Envía el comprobante de pago a nuestro WhatsApp
   • Indica el número de tu pedido en el mensaje
   • Tu pedido se preparará una vez confirmado el pago

🕐 Horario de atención: Lun-Vie 9:00-18:00
    """.strip()


# Crear FunctionTools para el agente
tools = [
    FunctionTool.from_defaults(fn=consultar_catalogo),
    FunctionTool.from_defaults(fn=calcular_precio),
    FunctionTool.from_defaults(fn=registrar_pedido),
    FunctionTool.from_defaults(fn=obtener_datos_transferencia),
]

print("✓ Tools del agente configuradas (4 tools disponibles)")

✓ Tools del agente configuradas (4 tools disponibles)


## 8. Crear Agente ReAct

In [14]:
from llama_index.core.agent.workflow import ReActAgent
from llama_index.core.workflow import Context

print("🤖 Creando agente ReAct...")

# Crear agente ReAct
# El agente usará las descripciones de las tools para decidir cuándo usarlas
agent = ReActAgent(
    tools=tools,
    llm=llm,
    verbose=True,
    timeout=120,  # timeout en segundos
)

# Crear contexto para mantener historial de conversación
ctx = Context(agent)

print("✓ Agente ReAct creado y configurado")

🤖 Creando agente ReAct...
✓ Agente ReAct creado y configurado


## 9. Interfaz de Chat Interactiva

In [15]:
import asyncio

async def chat_loop_async():
    """
    Loop de conversación interactiva con el agente (async).
    Escribe 'salir', 'exit', o 'quit' para terminar.
    """
    print("\n" + "="*60)
    print("🍰  PASTELERÍA DULCE TENTACIÓN - Asistente Virtual")
    print("="*60)
    print("\nEscribe 'salir' para terminar la conversación\n")

    while True:
        # Input del usuario
        user_input = input("\n👤 Tú: ").strip()

        # Salir
        if user_input.lower() in ['salir', 'exit', 'quit', 'chao', 'adios']:
            print("\n🍰 Agente: ¡Gracias por preferir Dulce Tentación! Que tengas un excelente día.\n")
            break

        # Ignorar inputs vacíos
        if not user_input:
            continue

        # Obtener respuesta del agente
        try:
            print("\n🤖 Agente está pensando...\n")

            # Ejecutar el agente con .run()
            handler = agent.run(user_input, ctx=ctx)

            from llama_index.core.agent.workflow import AgentStream

            async for ev in handler.stream_events():
                if isinstance(ev, AgentStream):
                    print(f"{ev.delta}", end="", flush=True)

            # Obtener la respuesta final
            response = await handler
            response_text = str(response)

            # Mostrar respuesta
            print(f"\n🍰 Agente: {response_text}\n")
            print("-" * 60)

        except Exception as e:
            print(f"\n❌ Error: {str(e)}\n")
            print("Por favor intenta de nuevo.\n")


await chat_loop_async()


🍰  PASTELERÍA DULCE TENTACIÓN - Asistente Virtual

Escribe 'salir' para terminar la conversación


👤 Tú: hola, quiero una torta de chocolate

🤖 Agente está pensando...

Thought: The current language of the user is: Spanish. I need to use a tool to help me answer the question.
Action: consultar_catalogo
Action Input: {"consulta": "torta chocolate"}Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: No hay un precio específico para una torta de chocolate en la información proporcionada. La opción más cercana es un cheesecake de chocolate que cuesta $30,000 para un tamaño que sirve a 10-12 personas.

Si necesitas una torta de chocolate específica, deberías preguntar directamente al pastelero o al vendedor para obtener el precio exacto de sus opciones de tortas de chocolate. Podrían ofrecerte una orden personalizada a un precio diferente.

¿Necesitas más detalles sobre alguna otra opción mencionada, como brownies o cupcakes?
🍰 Agente: No hay 

## 10. Visualizar Pedidos Registrados

In [16]:
import json
from pathlib import Path

def mostrar_pedidos():
    """Muestra todos los pedidos registrados en el sistema."""
    pedidos_file = Path("pedidos.json")

    if not pedidos_file.exists():
        print("No hay pedidos registrados aún.")
        return

    with open(pedidos_file, "r", encoding="utf-8") as f:
        pedidos = json.load(f)

    if not pedidos:
        print("No hay pedidos registrados aún.")
        return

    print("\n" + "="*60)
    print(f"PEDIDOS REGISTRADOS ({len(pedidos)} total)")
    print("="*60)

    for i, pedido in enumerate(pedidos, 1):
        print(f"\n📋 Pedido #{pedido['id']}")
        print(f"   Cliente: {pedido['cliente']['nombre']}")
        print(f"   Teléfono: {pedido['cliente']['telefono']}")
        print(f"   Dirección: {pedido['cliente']['direccion']}")
        print(f"   Items: {pedido['items']}")
        print(f"   Total: {pedido['total']}")
        print(f"   Estado: {pedido['estado']}")
        print(f"   Fecha: {pedido['timestamp']}")
        print("-" * 60)


mostrar_pedidos()


PEDIDOS REGISTRADOS (1 total)

📋 Pedido #20251126005554
   Cliente: Clemente
   Teléfono: 
   Dirección: 
   Items: torta chocolate 20cm
   Total: $25,000 CLP
   Estado: pendiente_pago
   Fecha: 2025-11-26T00:55:54.882179
------------------------------------------------------------


## 11. Ejercicios para Estudiantes

### Ejercicio 1: Agregar nueva tool
Crea una función `consultar_disponibilidad_fecha(fecha: str) -> str` que verifique
si hay disponibilidad para entregar un pedido en una fecha específica.

**Pistas:**
- Usa la librería `datetime` para parsear fechas
- Define días bloqueados (ej: festivos)
- Retorna mensaje indicando si está disponible

### Ejercicio 2: Sistema de descuentos
Crea una tool `aplicar_descuento(total: int, codigo: str) -> str` que:
- Valide códigos de descuento (ej: "PRIMERACOMPRA" = 10%)
- Calcule el nuevo total
- Retorne el desglose con el descuento aplicado

### Ejercicio 3: Mejorar validación de pedidos
Modifica `registrar_pedido` para validar que:
- El teléfono tenga formato chileno válido (+56 9 XXXX XXXX)
- La dirección incluya una comuna válida de Santiago
- El nombre tenga al menos 2 palabras
- Retorne errores descriptivos si falla la validación